# Boundary Relation Tests

## Setup

### Imports

In [61]:
# Shared Packages
import pandas as pd

# Local functions and classes
from types_and_classes import *
from utilities import *
from debug_tools import *
from structure_slice import *
from metrics import *
from relations import *

### Global Settings

In [ ]:
PRECISION = 2

In [63]:
%matplotlib inline

## Boundary Check process
1. Build slice table (index= slice, columns = ROI, Data= StructureSlice)
2. Select Primary & Secondary ROI
	- Slice range = Min(starting slice) to Max(ending slice)
3. Send all slices with both Primary and Secondary contours for standard relation testing 
4. Identify the boundary slices of the Primary and Secondary ROI
    - Boundary slices are slices that have a contour, but one of their neighbouring slices do not have a contour.
5. For each boundary slice of the Primary ROI identify the neighbouring slice(s) that do not have a primary.
6. For each of these neighbouring slices select a Secondary slice for boundary tests:
	- If the slice has a Secondary contour, select that Secondary slice.
	- If the slice does not have a Secondary contour, but there is a Secondary contour on the same slice as the Primary boundary, select that Secondary slice.
	- If neither the neighbouring slice nor the same slice as the Primary boundary have a Secondary contour, do not select a Secondary slice. Boundary testing is not required.
7. Test the relation between the boundary Primary and the selected Secondary.
8. Apply a Primary boundary shift to the relation results.
9. If the selected Secondary is also a Secondary boundary, apply a Secondary boundary shift as well.
10. Merge all results and reduce to single relation

## Test structures


#### Concentric cylinders starting on the same slice
  
<img src="Images\Boundaries\PartitionSup3D.png" alt="PartitionSup3D" style="height:50px;">
<img src="Images\Boundaries\PartitionSup2D.png" alt="PartitionSup2D" style="height:30px;">

In [64]:
def concentric_cylinders_same_start():
    slice_spacing = 0.1
    # Body structure defines slices in use
    body = make_vertical_cylinder(roi_num=0, radius=10, length=1, offset_z=-0.5,
                                  spacing=slice_spacing)
    # Concentric cylinders starting on the same slice
    primary_cylinder = make_vertical_cylinder(roi_num=1, radius=2, length=0.7,
                                              offset_z=-0.3,
                                              spacing=slice_spacing)
    sup_partition = make_vertical_cylinder(roi_num=2, radius=1, length=0.4,
                                           offset_z=-0.3,
                                           spacing=slice_spacing)
    # combine the contours
    slice_data = pd.concat([body, primary_cylinder, sup_partition])
    # convert contour slice data into a table of slices and structures
    slice_table = make_slice_table(slice_data, ignore_errors=True)
    return slice_table


In [65]:
#Build slice table (index= slice, columns = ROI, Data= StructureSlice)
slice_table = concentric_cylinders_same_start()

In [66]:
# Select Primary & Secondary ROI
selected_roi = [1,2]
# Slice range = Min(starting slice) to Max(ending slice)
selected_slices = select_slices(slice_table, selected_roi)


In [67]:
# Send all slices with both Primary and Secondary contours for standard relation testing
relation_seq = selected_slices.agg(relate_structures, structures=selected_roi,
                                  axis='columns')
relation_seq.name = 'DE9IM'

In [115]:
def match_neighbour_slices(slice_table, selected_roi):
    # For each boundary slice of the Primary ROI identify the neighbouring
    # slice(s) that do not have a primary.

    def get_neighbour_slices(boundary_index, slice_index, shift_dir)->pd.DataFrame:
        # Find the boundary neighbours
        # Get the index of the previous slice
        neighbour_slice = slice_index.shift(shift_dir)
        # Select only the slices that are boundary slices of the primary ROI
        neighbour_boundary_slices = neighbour_slice[boundary_index]
        # Drop the neighbour slices that contain a primary contour
        neighbour_boundary_slices.dropna(inplace=True)
        # Reset the index to get the boundary slice number
        neighbour_boundary_slices = neighbour_boundary_slices.reset_index()
        neighbour_boundary_slices.columns = ['Boundary', 'Neighbour']
        return neighbour_boundary_slices

    def no_structure_idx(slice_table, roi_num):
        # Create a series containing the slice index for slices that
        # do NOT have a primary contour
        no_contour_idx = slice_table.index.to_series(name='ROI_Index')
        # Select all slices that do not contain a contour for the Primary ROI
        missing_contour = slice_table[roi_num].apply(empty_structure)
        # Remove the slice indexes that have a primary contour
        no_contour_idx[~missing_contour] = np.nan
        return no_contour_idx

    primary, _ = selected_roi
    primary_boundaries = find_boundary_slices(slice_table[primary])

    # Get the slice index for slices that do NOT have a primary contour
    no_primary_slice_index = no_structure_idx(slice_table, primary)

    # Identify the previous and next slice for each boundary slice that do
    # not have a primary contour
    previous_slice = get_neighbour_slices(primary_boundaries,
                                          no_primary_slice_index, shift_dir=-1)
    next_slice = get_neighbour_slices(primary_boundaries,
                                      no_primary_slice_index, shift_dir=1)
    # Combine the previous and next slices
    neighbouring_slices = pd.concat([previous_slice, next_slice],
                                    ignore_index=True)
    return neighbouring_slices

In [70]:
primary, secondary = selected_roi


In [144]:
matched_slices = match_neighbour_slices(slice_table, selected_roi)
# If the slice has a Secondary contour, select that Secondary slice.
matched_slices = matched_slices.merge(slice_table[secondary],
                                      left_on='Neighbour',
                                      right_index=True, how='left')
#matched_slices.rename(columns={secondary:'Secondary'}, inplace=True)
# If the slice does not have a Secondary contour, but there is a Secondary
# contour on the same slice as the Primary boundary, select that
# Secondary slice.
same_slices = slice_table.loc[matched_slices.Boundary, secondary]
same_slices.index = matched_slices.index
no_neighbour = matched_slices[secondary].isnull()
matched_slices.loc[no_neighbour, secondary] = same_slices[no_neighbour]


# If neither the neighbouring slice nor the same slice as the Primary boundary
# have a Secondary contour, do not select a Secondary slice.
matched_slices.dropna(subset=[secondary], inplace=True)
# Select the Primary slice for each pair of Primary and Secondary slices
matched_slices = matched_slices.merge(slice_table[primary],
                                      left_on='Boundary',
                                      right_index=True, how='left')
#matched_slices.rename(columns={primary:'Primary'}, inplace=True)
matched_slices.drop(columns=['Boundary', 'Neighbour'], inplace=True)

In [150]:
# Test the relation between the boundary Primary and the selected Secondary.
relation_seq = matched_slices.agg(relate_structures, structures=selected_roi,
                                  axis='columns')
relation_seq

1    120034249
dtype: int64

In [ ]:
# Apply a Primary boundary shift to the relation results.


In [ ]:
# If the selected Secondary is also a Secondary boundary, apply a Secondary
#   boundary shift as well.
# Merge all results and reduce to single relation

In [ ]:

print(find_relationship(concentric_cylinders_same_start(), [1, 2]))


### Body Structure

In [63]:
slice_spacing = 0.1
# Body structure defines slices in use
body = make_vertical_cylinder(roi_num=0, radius=10, length=1,
                                         offset_z=-0.5, spacing=slice_spacing)


### Partition


In [64]:
# Two concentric cylinders different z offsets
primary_cylinder = make_vertical_cylinder(roi_num=1, radius=2, length=0.7,
                                         offset_z=-0.3, spacing=slice_spacing)


  - Concentric cylinders starting on the same slice
  
<img src="Images\Boundaries\PartitionSup3D.png" alt="PartitionSup3D" style="height:50px;">
<img src="Images\Boundaries\PartitionSup2D.png" alt="PartitionSup2D" style="height:30px;">

In [65]:
sup_partition = make_vertical_cylinder(roi_num=2, radius=1, length=0.4,
                                         offset_z=-0.3, spacing=slice_spacing)


- Concentric cylinders ending on the same slice.

<img src="Images\Boundaries\PartitionInf3D.png" alt="PartitionInf3D" style="height:50px;">
<img src="Images\Boundaries\PartitionInf2D.png" alt="PartitionInf2D" style="height:30px;">

In [66]:
inf_partition = make_vertical_cylinder(roi_num=3, radius=1, length=0.4,
                                         offset_z=0, spacing=slice_spacing)


- Concentric cylinders starting and ending on the same slice.

<img src="Images\Boundaries\Partition3D.png" alt="PartitionInf3D" style="height:50px;">
<img src="Images\Boundaries\Partition2D.png" alt="PartitionInf2D" style="height:30px;">

In [67]:

mid_partition = make_vertical_cylinder(roi_num=4, radius=1, length=0.7,
                                         offset_z=-0.3, spacing=slice_spacing)


### Exterior Borders
  - Primary: Central Cylinder
    - Secondary: one of:
      - SUP Cylinder
      - INF Cylinder
      - Combined SUP & INF cylinders in one structure with single slice gap at the SUP/INF boundary of the central cylinder.

      
![Exterior Border SUP](Images/Boundaries/ExteriorBorders2D_SUP.png)
![Exterior Border INF](Images/Boundaries/ExteriorBorders2D_INF.png)

In [68]:
# Two concentric cylinders different z offsets
outside_cylinder = make_vertical_cylinder(roi_num=5, radius=2, length=0.4,
                                         offset_z=-0.4, spacing=slice_spacing)
inside_cylinder = make_vertical_cylinder(roi_num=6, radius=1, length=0.4,
                                         offset_z=0, spacing=slice_spacing)


In [ ]:
# combine the contours
slice_data = pd.concat([body, primary_cylinder, sup_partition,
                        inf_partition, mid_partition,
                        outside_cylinder, inside_cylinder])

# convert contour slice data into a table of slices and structures
slice_table = make_slice_table(slice_data, ignore_errors=True)

In [70]:
selected_roi = [1, 2]
relation = process_relations(slice_table, selected_roi)
relation

C:\Users\smoke\AppData\Local\Temp\ipykernel_19644\3514096742.py:5: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  all_relations = pd.concat([mid_relations,


<RelationshipType.PARTITION: 7>

In [71]:
selected_roi = [1, 3]
relation = process_relations(slice_table, selected_roi)
relation


C:\Users\smoke\AppData\Local\Temp\ipykernel_19644\3514096742.py:5: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  all_relations = pd.concat([mid_relations,


<RelationshipType.PARTITION: 7>

In [72]:
selected_roi = [1, 4]
relation = process_relations(slice_table, selected_roi)
relation


C:\Users\smoke\AppData\Local\Temp\ipykernel_19644\3514096742.py:5: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  all_relations = pd.concat([mid_relations,


<RelationshipType.PARTITION: 7>

In [73]:
selected_roi = [5, 6]
relation = process_relations(slice_table, selected_roi)
relation


C:\Users\smoke\AppData\Local\Temp\ipykernel_19644\3514096742.py:5: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  all_relations = pd.concat([mid_relations,


<RelationshipType.BORDERS: 4>

# More Tests Needed

  - Single Primary slice with circular contour.
    - Secondary: one of:
      - SUP Cylinder
      - INF Cylinder
      - Combined SUP & INF Cylinders in one structure with single slice gap at level of the primary slice. 
- **Partition**
  - Concentric cylinders ending on the same slice.
  - Concentric cylinders starting on the same slice
  - Primary: Central Cylinder
    - Single Secondary slice with circular contour on the SUP/INF slice of the Primary cylinder.
  - Single Primary slice with circular contour
    - Secondary: one of:
      - SUP Cylinder ending on the same slice.
      - INF Cylinder ending on the same slice.
- **CONTAINS**
  - Concentric cylinders with interior cylinder ending inside the exterior cylinder by one slice
**OVERLAPS**
  - Concentric cylinders with interior cylinder ending outside the exterior cylinder by one slice
  - Concentric cylinders with interior cylinder consisting of single slice ending inside the exterior cylinder by one slice


  - Primary: Central Cylinder
    - Secondary: one of:
      - SUP Cylinder
      - INF Cylinder
      - Combined SUP & INF cylinders in one structure with single slice gap at the SUP/INF boundary of the central cylinder.
  - Single Primary slice with circular contour.
    - Secondary: one of:
      - SUP Cylinder
      - INF Cylinder
      - Combined SUP & INF Cylinders in one structure with single slice gap at level of the primary slice. 
